In [1]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

import matplotlib.cm as cm



In [2]:
pd.set_option('display.max_columns', 35)

In [4]:
SHEET_ID = '1gzQTlP9P4LtWXPgZOOAPsDsxAYbO70eCcHxyHmsl794'
DATA_TAB = 'DATA'
URL = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={DATA_TAB}'
df = pd.read_csv(URL)


df.columns = ['Team1 ', 'Score1 ', 'Team2 ', 'Score2 ', 'PERIOD ', 'Match ', 'Result ', 'WINNER ', 'LOSER ', 'H1 ', 'H2 ', 'Color ']

df.drop(['Color ', 'H1 ', 'H2 '], axis=1, inplace=True)
# filter out match NaN
df = df[df['Match '].notna()]
df['Match '] = df['Match '].astype(int)

df_swap = df.copy()
df_swap.columns = ['Team2 ', 'Score2 ', 'Team1 ', 'Score1 ', 'PERIOD ', 'Match ', 'Result ', 'WINNER ', 'LOSER ']

df_full = pd.concat([df, df_swap], axis=0)

df_full['WIN'] = np.where(df_full['Score1 '] > df_full['Score2 '], 1, 0)
df_full['WIN'] = np.where(df_full['Score1 '] == df_full['Score2 '], 0.5, df_full['WIN'])

# for each match number create cummulative sum of wins, score1 and score2 for each team
df_full1 = df_full.copy()
df_full1 = df_full1.sort_values(['Match ', 'Team1 '])


df_full1['WIN_CUMSUM'] = df_full1.groupby(['Team1 '])['WIN'].cumsum()
df_full1['SCORE1_CUMSUM'] = df_full1.groupby(['Team1 '])['Score1 '].cumsum()
df_full1['SCORE2_CUMSUM'] = df_full1.groupby(['Team1 '])['Score2 '].cumsum()

# select only the columns we need
df_full1 = df_full1[['Match ', 'Team1 ', 'WIN_CUMSUM', 'SCORE1_CUMSUM', 'SCORE2_CUMSUM']]

# create rank column within each week sorting by WIN_CUMSUM and then SCORE1_CUMSUM
df_full1 = df_full1.sort_values(['Match ', 'WIN_CUMSUM', 'SCORE1_CUMSUM'], ascending=[True, False, False])
df_full1['ranker'] = df_full1['WIN_CUMSUM']*1000000 + df_full1['SCORE1_CUMSUM']
df_full1['RANK'] = df_full1.groupby(['Match '])['ranker'].rank(ascending=False, method='first')#.astype(int)

df_full1['RANK'] = df_full1['RANK'].astype(int)

# df_full1[df_full1['Team1 '] == 'Memphis Grizzlies'].plot(x='Match ', y='RANK', kind='line', figsize=(15,5), title='Rank of Memphis Grizzlies over time')

# create dict of rankings for each team each week
rankings = {}
for team in df_full1['Team1 '].unique():
    rankings[team] = list(df_full1[df_full1['Team1 '] == team].set_index('Match ')['RANK'])

In [44]:
df_full

,Team1,Score1,Team2,Score2,PERIOD,Match,Result,WINNER,LOSER,WIN
0,Golden State Warriors,200.00,Brooklyn Nets,195.50,Scoring Period 1,1,FINAL,Golden State Warriors,Brooklyn Nets,1.0
1,Portland Trail Blazers,166.25,Atlanta Hawks,169.50,Scoring Period 1,1,FINAL,Atlanta Hawks,Portland Trail Blazers,0.0
2,Houston Rockets,206.50,Philadelphia 76ers,166.75,Scoring Period 1,1,FINAL,Houston Rockets,Philadelphia 76ers,1.0
3,Oklahoma City Thunder,239.75,Minnesota Timberwolves,288.50,Scoring Period 1,1,FINAL,Minnesota Timberwolves,Oklahoma City Thunder,0.0
4,Toronto Raptors,213.50,Sacramento Kings,189.75,Scoring Period 1,1,FINAL,Toronto Raptors,Sacramento Kings,1.0
...,...,...,...,...,...,...,...,...,...,...
208,Memphis Grizzlies,419.50,Portland Trail Blazers,433.50,Scoring Period 12,12,FINAL,Portland Trail Blazers,Memphis Grizzlies,0.0
209,Dallas Mavericks,385.75,Golden State Warriors,392.75,Scoring Period 12,12,FINAL,Golden State Warriors,Dallas Mavericks,0.0
210,Miami Heat,272.00,Atlanta Hawks,457.00,Scoring Period 12,12,FINAL,Atlanta Hawks,Miami Heat,0.0
211,Milwaukee Bucks,80.00,Philadelphia 76ers,312.00,Scoring Period 12,12,FINAL,Philadelphia 76ers,Milwaukee Bucks,0.0


In [9]:
df_scores = df_full.groupby(['Team1 '])[['Score1 ', 'Score2 ']].sum().reset_index()

df_scores.columns = ['Team', 'Scored For', 'Scored Against']
median_scored_for = np.median(df_scores['Scored For'])
median_scored_against = np.median(df_scores['Scored Against'])

In [10]:
df_scores

,Team,Scored For,Scored Against
0,Atlanta Hawks,4938.00,4110.50
1,Boston Celtics,4444.75,4676.50
2,Brooklyn Nets,5093.50,4524.75
3,Charlotte Hornets,5546.50,4754.25
4,Chicago Bulls,4181.50,4782.75
5,Cleveland Cavaliers,3228.00,4303.50
6,Dallas Mavericks,4875.50,3946.50
7,Denver Nuggets,4482.75,4032.75
8,Detroit Pistons,5292.25,4817.25
9,Golden State Warriors,4478.25,4532.50


In [8]:
rankings


{'Chicago Bulls': [1, 14, 7, 5, 7, 6, 9, 11, 14, 13, 17, 18],
 'Dallas Mavericks': [2, 4, 6, 9, 14, 17, 14, 15, 13, 11, 10, 12],
 'Utah Jazz': [3, 2, 3, 3, 3, 3, 2, 2, 3, 5, 6, 5],
 'Minnesota Timberwolves': [4, 6, 4, 4, 4, 5, 5, 4, 7, 6, 4, 7],
 'Charlotte Hornets': [5, 9, 15, 12, 9, 7, 6, 5, 4, 7, 8, 8],
 'Orlando Magic': [6, 3, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1],
 'Memphis Grizzlies': [7, 7, 12, 10, 8, 11, 8, 8, 8, 9, 11, 15],
 'Detroit Pistons': [8, 1, 1, 1, 2, 2, 3, 3, 2, 3, 5, 4],
 'Cleveland Cavaliers': [9, 17, 11, 19, 20, 24, 26, 27, 28, 29, 25, 27],
 'Denver Nuggets': [10, 19, 22, 22, 18, 14, 19, 17, 19, 16, 15, 13],
 'Toronto Raptors': [11, 5, 5, 7, 6, 10, 7, 7, 6, 4, 3, 3],
 'Washington Wizards': [12, 22, 25, 26, 28, 27, 23, 24, 25, 25, 27, 29],
 'Houston Rockets': [13, 8, 14, 11, 15, 18, 20, 22, 22, 22, 22, 20],
 'Golden State Warriors': [14, 20, 24, 25, 25, 22, 22, 20, 18, 17, 16, 14],
 'Atlanta Hawks': [15, 21, 13, 18, 13, 16, 15, 10, 9, 8, 7, 6],
 'New Orleans Pelicans': [16,

In [ ]:
# for each match number create cummulative sum of wins, score1 and score2 for each team
df_full1 = df_full.copy()
df_full1 = df_full1.sort_values(['Match ', 'Team1 '])

df_full1['WIN_CUMSUM'] = df_full1.groupby(['Team1 '])['WIN'].cumsum()
df_full1['SCORE1_CUMSUM'] = df_full1.groupby(['Team1 '])['Score1 '].cumsum()
df_full1['SCORE2_CUMSUM'] = df_full1.groupby(['Team1 '])['Score2 '].cumsum()

# select only the columns we need
df_full1 = df_full1[['Match ', 'Team1 ', 'WIN_CUMSUM', 'SCORE1_CUMSUM', 'SCORE2_CUMSUM']]

# create rank column within each week sorting by WIN_CUMSUM and then SCORE1_CUMSUM
df_full1 = df_full1.sort_values(['Match ', 'WIN_CUMSUM', 'SCORE1_CUMSUM'], ascending=[True, False, False])
df_full1['ranker'] = df_full1['WIN_CUMSUM'] * 1000000 + df_full1['SCORE1_CUMSUM']
df_full1['RANK'] = df_full1.groupby(['Match '])['ranker'].rank(ascending=False, method='first')  # .astype(int)

df_full1['RANK'] = df_full1['RANK'].astype(int)

# create dict of rankings for each team each week
rankings = {}
for team in df_full1['Team1 '].unique():
    rankings[team] = list(df_full1[df_full1['Team1 '] == team].set_index('Match ')['RANK'])

In [3]:
SHEET_ID = '1gzQTlP9P4LtWXPgZOOAPsDsxAYbO70eCcHxyHmsl794'
DATA_TAB = 'DATA'
URL = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={DATA_TAB}'
df = pd.read_csv(URL)


df.columns = ['Team1 ', 'Score1 ', 'Team2 ', 'Score2 ', 'PERIOD ', 'Match ', 'Result ', 'WINNER ', 'LOSER ', 'H1 ', 'H2 ', 'Color ']

df.drop(['Color ', 'H1 ', 'H2 '], axis=1, inplace=True)
# filter out match NaN
df = df[df['Match '].notna()]
df['Match '] = df['Match '].astype(int)

df_swap = df.copy()
df_swap.columns = ['Team2 ', 'Score2 ', 'Team1 ', 'Score1 ', 'PERIOD ', 'Match ', 'Result ', 'WINNER ', 'LOSER ']

df_full = pd.concat([df, df_swap], axis=0)

df_full['WIN'] = np.where(df_full['Score1 '] > df_full['Score2 '], 1, 0)
df_full['WIN'] = np.where(df_full['Score1 '] == df_full['Score2 '], 0.5, df_full['WIN'])

In [4]:
df_full

,Team1,Score1,Team2,Score2,PERIOD,Match,Result,WINNER,LOSER,WIN
0,Golden State Warriors,200.00,Brooklyn Nets,195.50,Scoring Period 1,1,FINAL,Golden State Warriors,Brooklyn Nets,1.0
1,Portland Trail Blazers,166.25,Atlanta Hawks,169.50,Scoring Period 1,1,FINAL,Atlanta Hawks,Portland Trail Blazers,0.0
2,Houston Rockets,206.50,Philadelphia 76ers,166.75,Scoring Period 1,1,FINAL,Houston Rockets,Philadelphia 76ers,1.0
3,Oklahoma City Thunder,239.75,Minnesota Timberwolves,288.50,Scoring Period 1,1,FINAL,Minnesota Timberwolves,Oklahoma City Thunder,0.0
4,Toronto Raptors,213.50,Sacramento Kings,189.75,Scoring Period 1,1,FINAL,Toronto Raptors,Sacramento Kings,1.0
...,...,...,...,...,...,...,...,...,...,...
208,Memphis Grizzlies,419.50,Portland Trail Blazers,433.50,Scoring Period 12,12,FINAL,Portland Trail Blazers,Memphis Grizzlies,0.0
209,Dallas Mavericks,385.75,Golden State Warriors,392.75,Scoring Period 12,12,FINAL,Golden State Warriors,Dallas Mavericks,0.0
210,Miami Heat,272.00,Atlanta Hawks,457.00,Scoring Period 12,12,FINAL,Atlanta Hawks,Miami Heat,0.0
211,Milwaukee Bucks,80.00,Philadelphia 76ers,312.00,Scoring Period 12,12,FINAL,Philadelphia 76ers,Milwaukee Bucks,0.0


In [22]:
df_alter = df_full.copy()

#select only the columns we need
df_alter = df_alter[['Match ', 'Team1 ', 'Score1 ']]
df_alter.columns = ['Match ', 'Alt_Team', 'Alt_Score']
df_alter


,Match,Alt_Team,Alt_Score
0,1,Golden State Warriors,200.00
1,1,Portland Trail Blazers,166.25
2,1,Houston Rockets,206.50
3,1,Oklahoma City Thunder,239.75
4,1,Toronto Raptors,213.50
...,...,...,...
208,12,Memphis Grizzlies,419.50
209,12,Dallas Mavericks,385.75
210,12,Miami Heat,272.00
211,12,Milwaukee Bucks,80.00


In [23]:
df_full.columns

Index(['Team1 ', 'Score1 ', 'Team2 ', 'Score2 ', 'PERIOD ', 'Match ',
       'Result ', 'WINNER ', 'LOSER ', 'WIN'],
      dtype='object')

In [46]:
df_wins = pd.pivot_table(df_full, values='WIN', index=['Team1 '], columns=['Team2 '], aggfunc='sum')

# replace 0 with -1
# df_wins = df_wins.replace(0, -1)
# df_wins = df_wins.replace(0.5, 0)

#fill nan with 0
df_wins = df_wins.fillna(-1)

df_wins


Team2,Atlanta Hawks,Boston Celtics,Brooklyn Nets,Charlotte Hornets,Chicago Bulls,Cleveland Cavaliers,Dallas Mavericks,Denver Nuggets,Detroit Pistons,Golden State Warriors,Houston Rockets,Indiana Pacers,Los Angeles Clippers,Los Angeles Lakers,Memphis Grizzlies,Miami Heat,Milwaukee Bucks,Minnesota Timberwolves,New Orleans Pelicans,New York Knicks,Oklahoma City Thunder,Orlando Magic,Philadelphia 76ers,Phoenix Suns,Portland Trail Blazers,Sacramento Kings,San Antonio Spurs,Toronto Raptors,Utah Jazz,Washington Wizards
Team1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Atlanta Hawks,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN
Boston Celtics,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,1.0,NaN,0.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN
Brooklyn Nets,NaN,NaN,NaN,NaN,NaN,1.0,NaN,0.0,NaN,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,1.0
Charlotte Hornets,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,1.0,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,1.0,1.0,NaN,NaN
Chicago Bulls,0.0,1.0,NaN,1.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,0.0,0.0,NaN,1.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
Cleveland Cavaliers,0.0,1.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
Dallas Mavericks,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,0.0,0.0,NaN,NaN,1.0,NaN,0.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN
Denver Nuggets,0.0,NaN,1.0,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,0.0,NaN,1.0,NaN,NaN,NaN
Detroit Pistons,0.0,1.0,NaN,1.0,NaN,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN


In [47]:
df_alt_merge = df_full.merge(df_alter, on='Match ', how = 'inner')
df_alt_merge['Alt_WIN'] = np.where(df_alt_merge['Score1 '] > df_alt_merge['Alt_Score'], 1, 0)
df_alt_merge['Alt_WIN'] = np.where(df_alt_merge['Score1 '] == df_alt_merge['Alt_Score'], 0.5, df_alt_merge['Alt_WIN'])

# remove rows where Team1 = Alt_Team
df_alt_merge = df_alt_merge[df_alt_merge['Team1 '] != df_alt_merge['Alt_Team']]

#filter by team Memphis Grizzlies and match number = 1
# df_alt_merge[(df_alt_merge['Team1 '] == 'Memphis Grizzlies') & (df_alt_merge['Match '] == 1)]

df_alt_wins = pd.pivot_table(df_alt_merge, values='Alt_WIN', index=['Team1 '], columns=['Alt_Team'], aggfunc='sum')

#divide by number of games played between each team
df_alt_wins = df_alt_wins.div(pd.pivot_table(df_alt_merge, values='Alt_WIN', index=['Team1 '], columns=['Alt_Team'], aggfunc='count'), axis=1)

#round up to 2 decimal places
df_alt_wins = df_alt_wins.round(2)


df_alt_wins




Alt_Team,Atlanta Hawks,Boston Celtics,Brooklyn Nets,Charlotte Hornets,Chicago Bulls,Cleveland Cavaliers,Dallas Mavericks,Denver Nuggets,Detroit Pistons,Golden State Warriors,Houston Rockets,Indiana Pacers,Los Angeles Clippers,Los Angeles Lakers,Memphis Grizzlies,Miami Heat,Milwaukee Bucks,Minnesota Timberwolves,New Orleans Pelicans,New York Knicks,Oklahoma City Thunder,Orlando Magic,Philadelphia 76ers,Phoenix Suns,Portland Trail Blazers,Sacramento Kings,San Antonio Spurs,Toronto Raptors,Utah Jazz,Washington Wizards,Strength,Rank
Team1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Atlanta Hawks,NaN,0.75,0.50,0.25,0.67,0.83,0.58,0.58,0.42,0.67,0.67,0.33,0.75,0.75,0.67,0.75,1.00,0.58,0.33,0.75,0.58,0.25,1.00,0.58,0.42,0.42,0.83,0.42,0.50,0.62,0.601724,11.0
Boston Celtics,0.25,NaN,0.33,0.00,0.58,0.83,0.42,0.50,0.08,0.50,0.42,0.17,0.67,0.58,0.33,0.83,1.00,0.33,0.17,0.58,0.58,0.08,0.83,0.25,0.08,0.33,0.67,0.08,0.17,0.50,0.418621,22.0
Brooklyn Nets,0.50,0.67,NaN,0.42,0.67,0.92,0.58,0.58,0.50,0.58,0.58,0.67,0.92,0.67,0.58,0.92,1.00,0.58,0.25,0.75,0.58,0.25,0.92,0.50,0.42,0.50,0.67,0.25,0.54,0.75,0.611034,10.0
Charlotte Hornets,0.75,1.00,0.58,NaN,0.75,1.00,0.75,0.83,0.75,0.83,0.92,0.75,1.00,0.83,0.92,1.00,1.00,0.75,0.50,0.92,1.00,0.42,1.00,0.83,0.67,0.67,1.00,0.75,0.67,1.00,0.822069,2.0
Chicago Bulls,0.33,0.42,0.33,0.25,NaN,0.75,0.50,0.42,0.25,0.42,0.50,0.25,0.58,0.50,0.42,0.58,0.92,0.25,0.25,0.67,0.42,0.25,0.58,0.42,0.17,0.33,0.75,0.25,0.25,0.58,0.434138,19.0
Cleveland Cavaliers,0.17,0.17,0.08,0.00,0.25,NaN,0.08,0.25,0.00,0.25,0.33,0.08,0.42,0.08,0.17,0.58,0.83,0.00,0.08,0.42,0.25,0.00,0.50,0.17,0.08,0.08,0.50,0.08,0.00,0.33,0.214828,28.0
Dallas Mavericks,0.42,0.58,0.42,0.25,0.50,0.92,NaN,0.67,0.42,0.50,0.58,0.42,0.92,0.67,0.58,0.83,1.00,0.58,0.33,0.83,0.50,0.33,0.83,0.58,0.25,0.25,0.75,0.25,0.33,0.67,0.557241,13.0
Denver Nuggets,0.42,0.50,0.42,0.17,0.58,0.75,0.33,NaN,0.33,0.67,0.67,0.25,0.67,0.67,0.42,0.83,0.83,0.42,0.17,0.75,0.67,0.25,0.75,0.50,0.33,0.42,0.79,0.42,0.33,0.67,0.516552,15.0
Detroit Pistons,0.58,0.92,0.50,0.25,0.75,1.00,0.58,0.67,NaN,0.67,0.83,0.50,0.92,0.83,0.67,1.00,1.00,0.67,0.42,0.92,1.00,0.33,1.00,0.83,0.42,0.58,0.92,0.54,0.50,0.92,0.714483,6.0


In [48]:
df_alt_wins['Strength'] = df_alt_wins.mean(axis=1)

# add rank column
df_alt_wins['Rank'] = df_alt_wins['Strength'].rank(ascending=False).astype(int)

# Fill NaN values with -1 to indicate "not applicable"
df_alt_wins = df_alt_wins.fillna(-1)

# Sort teams alphabetically
teams = sorted([col for col in df_alt_wins.columns if col not in ['Strength', 'Rank']])

# Reorder columns to match sorted teams (plus Strength and Rank at the end)
df_alt_wins = df_alt_wins[teams + ['Strength', 'Rank']]

In [49]:
df_alt_wins

Alt_Team,Atlanta Hawks,Boston Celtics,Brooklyn Nets,Charlotte Hornets,Chicago Bulls,Cleveland Cavaliers,Dallas Mavericks,Denver Nuggets,Detroit Pistons,Golden State Warriors,Houston Rockets,Indiana Pacers,Los Angeles Clippers,Los Angeles Lakers,Memphis Grizzlies,Miami Heat,Milwaukee Bucks,Minnesota Timberwolves,New Orleans Pelicans,New York Knicks,Oklahoma City Thunder,Orlando Magic,Philadelphia 76ers,Phoenix Suns,Portland Trail Blazers,Sacramento Kings,San Antonio Spurs,Toronto Raptors,Utah Jazz,Washington Wizards,Strength,Rank
Team1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Atlanta Hawks,-1.00,0.75,0.50,0.25,0.67,0.83,0.58,0.58,0.42,0.67,0.67,0.33,0.75,0.75,0.67,0.75,1.00,0.58,0.33,0.75,0.58,0.25,1.00,0.58,0.42,0.42,0.83,0.42,0.50,0.62,0.601724,11.0
Boston Celtics,0.25,-1.00,0.33,0.00,0.58,0.83,0.42,0.50,0.08,0.50,0.42,0.17,0.67,0.58,0.33,0.83,1.00,0.33,0.17,0.58,0.58,0.08,0.83,0.25,0.08,0.33,0.67,0.08,0.17,0.50,0.418621,22.0
Brooklyn Nets,0.50,0.67,-1.00,0.42,0.67,0.92,0.58,0.58,0.50,0.58,0.58,0.67,0.92,0.67,0.58,0.92,1.00,0.58,0.25,0.75,0.58,0.25,0.92,0.50,0.42,0.50,0.67,0.25,0.54,0.75,0.611034,10.0
Charlotte Hornets,0.75,1.00,0.58,-1.00,0.75,1.00,0.75,0.83,0.75,0.83,0.92,0.75,1.00,0.83,0.92,1.00,1.00,0.75,0.50,0.92,1.00,0.42,1.00,0.83,0.67,0.67,1.00,0.75,0.67,1.00,0.822069,2.0
Chicago Bulls,0.33,0.42,0.33,0.25,-1.00,0.75,0.50,0.42,0.25,0.42,0.50,0.25,0.58,0.50,0.42,0.58,0.92,0.25,0.25,0.67,0.42,0.25,0.58,0.42,0.17,0.33,0.75,0.25,0.25,0.58,0.434138,19.0
Cleveland Cavaliers,0.17,0.17,0.08,0.00,0.25,-1.00,0.08,0.25,0.00,0.25,0.33,0.08,0.42,0.08,0.17,0.58,0.83,0.00,0.08,0.42,0.25,0.00,0.50,0.17,0.08,0.08,0.50,0.08,0.00,0.33,0.214828,28.0
Dallas Mavericks,0.42,0.58,0.42,0.25,0.50,0.92,-1.00,0.67,0.42,0.50,0.58,0.42,0.92,0.67,0.58,0.83,1.00,0.58,0.33,0.83,0.50,0.33,0.83,0.58,0.25,0.25,0.75,0.25,0.33,0.67,0.557241,13.0
Denver Nuggets,0.42,0.50,0.42,0.17,0.58,0.75,0.33,-1.00,0.33,0.67,0.67,0.25,0.67,0.67,0.42,0.83,0.83,0.42,0.17,0.75,0.67,0.25,0.75,0.50,0.33,0.42,0.79,0.42,0.33,0.67,0.516552,15.0
Detroit Pistons,0.58,0.92,0.50,0.25,0.75,1.00,0.58,0.67,-1.00,0.67,0.83,0.50,0.92,0.83,0.67,1.00,1.00,0.67,0.42,0.92,1.00,0.33,1.00,0.83,0.42,0.58,0.92,0.54,0.50,0.92,0.714483,6.0


In [5]:

SHEET_ID = '1gzQTlP9P4LtWXPgZOOAPsDsxAYbO70eCcHxyHmsl794'
DATA_TAB = 'DATA'
URL = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={DATA_TAB}'
df = pd.read_csv(URL)


df.columns = ['Team1 ', 'Score1 ', 'Team2 ', 'Score2 ', 'PERIOD ', 'Match ', 'Result ', 'WINNER ', 'LOSER ', 'H1 ', 'H2 ', 'Color ']

df.drop(['Color ', 'H1 ', 'H2 '], axis=1, inplace=True)
# filter out match NaN
df = df[df['Match '].notna()]
df['Match '] = df['Match '].astype(int)

df_swap = df.copy()
df_swap.columns = ['Team2 ', 'Score2 ', 'Team1 ', 'Score1 ', 'PERIOD ', 'Match ', 'Result ', 'WINNER ', 'LOSER ']

df_full = pd.concat([df, df_swap], axis=0)

df_full['WIN'] = np.where(df_full['Score1 '] > df_full['Score2 '], 1, 0)
df_full['WIN'] = np.where(df_full['Score1 '] == df_full['Score2 '], 0.5, df_full['WIN'])

df_wins = pd.pivot_table(df_full, values='WIN', index=['Team1 '], columns=['Team2 '], aggfunc='sum')

df_alter = df_full.copy()

#select only the columns we need
df_alter = df_alter[['Match ', 'Team1 ', 'Score1 ']]
df_alter.columns = ['Match ', 'Alt_Team', 'Alt_Score']
df_alter


df_alt_merge = df_full.merge(df_alter, on='Match ', how = 'inner')
df_alt_merge['Alt_WIN'] = np.where(df_alt_merge['Score1 '] > df_alt_merge['Alt_Score'], 1, 0)
df_alt_merge['Alt_WIN'] = np.where(df_alt_merge['Score1 '] == df_alt_merge['Alt_Score'], 0.5, df_alt_merge['Alt_WIN'])

# remove rows where Team1 = Alt_Team
df_alt_merge = df_alt_merge[df_alt_merge['Team1 '] != df_alt_merge['Alt_Team']]


df_alt_wins = pd.pivot_table(df_alt_merge, values='Alt_WIN', index=['Team1 '], columns=['Alt_Team'], aggfunc='sum')

#divide by number of games played between each team
df_alt_wins = df_alt_wins.div(pd.pivot_table(df_alt_merge, values='Alt_WIN', index=['Team1 '], columns=['Alt_Team'], aggfunc='count'), axis=1)

#round up to 2 decimal places
df_alt_wins = df_alt_wins.round(2)

In [6]:
df_wins

Team2,Atlanta Hawks,Boston Celtics,Brooklyn Nets,Charlotte Hornets,Chicago Bulls,Cleveland Cavaliers,Dallas Mavericks,Denver Nuggets,Detroit Pistons,Golden State Warriors,Houston Rockets,Indiana Pacers,Los Angeles Clippers,Los Angeles Lakers,Memphis Grizzlies,Miami Heat,Milwaukee Bucks,Minnesota Timberwolves,New Orleans Pelicans,New York Knicks,Oklahoma City Thunder,Orlando Magic,Philadelphia 76ers,Phoenix Suns,Portland Trail Blazers,Sacramento Kings,San Antonio Spurs,Toronto Raptors,Utah Jazz,Washington Wizards
Team1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Atlanta Hawks,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,1.0,0.0,1.0,NaN,NaN,NaN
Boston Celtics,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,1.0,NaN,0.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN
Brooklyn Nets,NaN,NaN,NaN,NaN,NaN,1.0,NaN,0.0,NaN,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,0.0,1.0,1.0
Charlotte Hornets,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,1.0,NaN,1.0,1.0,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN,1.0,1.0,1.0,NaN,NaN
Chicago Bulls,0.0,1.0,NaN,1.0,NaN,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,0.0,0.0,NaN,1.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
Cleveland Cavaliers,0.0,1.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
Dallas Mavericks,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,0.0,0.0,1.0,NaN,1.0,NaN,0.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN
Denver Nuggets,0.0,NaN,1.0,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,0.0,NaN,1.0,NaN,NaN,NaN
Detroit Pistons,0.0,1.0,NaN,1.0,NaN,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN


In [7]:
df_alt_wins

Alt_Team,Atlanta Hawks,Boston Celtics,Brooklyn Nets,Charlotte Hornets,Chicago Bulls,Cleveland Cavaliers,Dallas Mavericks,Denver Nuggets,Detroit Pistons,Golden State Warriors,Houston Rockets,Indiana Pacers,Los Angeles Clippers,Los Angeles Lakers,Memphis Grizzlies,Miami Heat,Milwaukee Bucks,Minnesota Timberwolves,New Orleans Pelicans,New York Knicks,Oklahoma City Thunder,Orlando Magic,Philadelphia 76ers,Phoenix Suns,Portland Trail Blazers,Sacramento Kings,San Antonio Spurs,Toronto Raptors,Utah Jazz,Washington Wizards
Team1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Atlanta Hawks,NaN,0.77,0.54,0.31,0.69,0.85,0.54,0.62,0.46,0.69,0.69,0.31,0.77,0.77,0.62,0.77,1.00,0.62,0.31,0.77,0.62,0.31,1.00,0.62,0.46,0.38,0.85,0.46,0.54,0.65
Boston Celtics,0.23,NaN,0.31,0.00,0.54,0.77,0.38,0.46,0.08,0.46,0.46,0.15,0.69,0.54,0.31,0.77,1.00,0.38,0.15,0.54,0.54,0.15,0.77,0.23,0.08,0.31,0.69,0.08,0.15,0.46
Brooklyn Nets,0.46,0.69,NaN,0.38,0.62,0.92,0.54,0.54,0.46,0.62,0.62,0.62,0.92,0.69,0.54,0.92,1.00,0.62,0.23,0.77,0.62,0.31,0.92,0.54,0.38,0.46,0.69,0.31,0.58,0.77
Charlotte Hornets,0.69,1.00,0.62,NaN,0.77,1.00,0.69,0.77,0.77,0.85,0.92,0.69,1.00,0.85,0.85,1.00,1.00,0.77,0.46,0.92,1.00,0.46,1.00,0.85,0.69,0.62,1.00,0.77,0.69,1.00
Chicago Bulls,0.31,0.46,0.38,0.23,NaN,0.77,0.46,0.38,0.23,0.46,0.54,0.23,0.62,0.54,0.38,0.62,0.92,0.31,0.23,0.69,0.46,0.31,0.62,0.46,0.15,0.31,0.77,0.31,0.31,0.62
Cleveland Cavaliers,0.15,0.23,0.08,0.00,0.23,NaN,0.08,0.23,0.00,0.23,0.38,0.08,0.46,0.15,0.15,0.54,0.85,0.08,0.08,0.38,0.31,0.08,0.46,0.23,0.08,0.08,0.54,0.15,0.00,0.31
Dallas Mavericks,0.46,0.62,0.46,0.31,0.54,0.92,NaN,0.69,0.46,0.54,0.62,0.46,0.92,0.69,0.62,0.85,1.00,0.62,0.38,0.85,0.54,0.38,0.85,0.62,0.31,0.31,0.77,0.31,0.38,0.69
Denver Nuggets,0.38,0.54,0.46,0.23,0.62,0.77,0.31,NaN,0.38,0.69,0.69,0.23,0.69,0.69,0.38,0.85,0.85,0.46,0.15,0.77,0.69,0.31,0.77,0.54,0.38,0.38,0.81,0.46,0.38,0.69
Detroit Pistons,0.54,0.92,0.54,0.23,0.77,1.00,0.54,0.62,NaN,0.69,0.85,0.46,0.92,0.85,0.62,1.00,1.00,0.69,0.38,0.92,1.00,0.38,1.00,0.85,0.46,0.54,0.92,0.58,0.54,0.92


In [8]:
# subtract df_alt_wins from df_wins
df_diff = df_wins - df_alt_wins

df_diff['Strength'] = df_diff.mean(axis=1)

# add rank column
df_diff['Rank'] = df_diff['Strength'].rank(ascending=False).astype(int)

# Fill NaN values with -1 to indicate "not applicable"
df_diff = df_diff.fillna(-1)

# Sort teams alphabetically
teams = sorted([col for col in df_diff.columns if col not in ['Strength', 'Rank']])

# Reorder columns to match sorted teams (plus Strength and Rank at the end)
df_diff = df_diff[teams + ['Strength', 'Rank']]

df_diff

Team2,Atlanta Hawks,Boston Celtics,Brooklyn Nets,Charlotte Hornets,Chicago Bulls,Cleveland Cavaliers,Dallas Mavericks,Denver Nuggets,Detroit Pistons,Golden State Warriors,Houston Rockets,Indiana Pacers,Los Angeles Clippers,Los Angeles Lakers,Memphis Grizzlies,Miami Heat,Milwaukee Bucks,Minnesota Timberwolves,New Orleans Pelicans,New York Knicks,Oklahoma City Thunder,Orlando Magic,Philadelphia 76ers,Phoenix Suns,Portland Trail Blazers,Sacramento Kings,San Antonio Spurs,Toronto Raptors,Utah Jazz,Washington Wizards
Team1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Atlanta Hawks,NaN,NaN,NaN,NaN,0.31,0.15,NaN,0.38,0.54,NaN,NaN,NaN,0.23,NaN,0.38,0.23,NaN,NaN,-0.31,-0.77,-0.62,NaN,NaN,NaN,0.54,-0.38,0.15,NaN,NaN,NaN
Boston Celtics,NaN,NaN,NaN,NaN,-0.54,-0.77,NaN,NaN,-0.08,-0.46,-0.46,NaN,0.31,NaN,-0.31,0.23,NaN,NaN,NaN,0.46,NaN,NaN,0.23,NaN,NaN,-0.31,-0.69,-0.08,NaN,NaN
Brooklyn Nets,NaN,NaN,NaN,NaN,NaN,0.08,NaN,-0.54,NaN,-0.62,0.38,0.38,NaN,NaN,NaN,NaN,NaN,NaN,0.77,0.23,-0.62,-0.31,NaN,NaN,-0.38,NaN,NaN,-0.31,0.42,0.23
Charlotte Hornets,NaN,NaN,NaN,NaN,-0.77,NaN,NaN,NaN,-0.77,-0.85,-0.92,NaN,0.00,NaN,0.15,0.00,NaN,NaN,NaN,0.08,NaN,0.54,0.00,NaN,NaN,0.38,0.00,0.23,NaN,NaN
Chicago Bulls,-0.31,0.54,NaN,0.77,NaN,0.23,0.54,-0.38,NaN,NaN,NaN,NaN,NaN,0.46,NaN,NaN,0.08,-0.31,-0.23,NaN,0.54,NaN,NaN,-0.46,-0.15,NaN,NaN,NaN,NaN,NaN
Cleveland Cavaliers,-0.15,0.77,-0.08,NaN,-0.23,NaN,-0.08,-0.23,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.15,-0.08,-0.08,0.62,-0.31,NaN,NaN,NaN,-0.08,NaN,NaN,NaN,NaN,NaN
Dallas Mavericks,NaN,NaN,NaN,NaN,-0.54,0.08,NaN,NaN,-0.46,-0.54,0.38,NaN,0.08,NaN,-0.62,0.15,NaN,NaN,0.62,0.15,NaN,NaN,0.15,NaN,NaN,0.69,-0.77,NaN,NaN,NaN
Denver Nuggets,-0.38,NaN,0.54,NaN,0.38,0.23,NaN,NaN,0.62,NaN,NaN,NaN,0.31,NaN,NaN,NaN,0.15,-0.46,-0.15,-0.77,0.31,NaN,NaN,NaN,-0.38,NaN,0.19,NaN,NaN,NaN
Detroit Pistons,-0.54,0.08,NaN,0.77,NaN,0.00,0.46,-0.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.31,-0.38,0.08,0.00,NaN,NaN,0.15,0.54,NaN,NaN,NaN,NaN,NaN


In [9]:
# sum of all values in df_diff per row
df_diff.sum(axis=1)


Team1 
Atlanta Hawks             0.83
Boston Celtics           -2.47
Brooklyn Nets            -0.29
Charlotte Hornets        -1.93
Chicago Bulls             1.32
Cleveland Cavaliers       0.22
Dallas Mavericks         -0.63
Denver Nuggets            0.59
Detroit Pistons           0.85
Golden State Warriors     2.28
Houston Rockets           0.25
Indiana Pacers           -0.54
Los Angeles Clippers     -0.44
Los Angeles Lakers       -2.29
Memphis Grizzlies         1.87
Miami Heat               -0.07
Milwaukee Bucks          -0.61
Minnesota Timberwolves   -0.01
New Orleans Pelicans     -1.08
New York Knicks          -0.78
Oklahoma City Thunder    -0.44
Orlando Magic             1.75
Philadelphia 76ers        1.02
Phoenix Suns             -0.70
Portland Trail Blazers    1.13
Sacramento Kings          0.17
San Antonio Spurs        -0.03
Toronto Raptors           1.37
Utah Jazz                 0.25
Washington Wizards       -1.59
dtype: float64

In [10]:
df_full = pd.concat([df, df_swap], axis=0)

df_full['WIN'] = np.where(df_full['Score1 '] > df_full['Score2 '], 1, 0)
df_full['WIN'] = np.where(df_full['Score1 '] == df_full['Score2 '], 0.5, df_full['WIN'])

df_wins = pd.pivot_table(df_full, values='WIN', index=['Team1 '], columns=['Team2 '], aggfunc='sum')
df_wins = df_wins.fillna(-1)

df_alter = df_full.copy()

# select only the columns we need
df_alter = df_alter[['Match ', 'Team1 ', 'Score1 ']]
df_alter.columns = ['Match ', 'Alt_Team', 'Alt_Score']

df_alt_merge = df_full.merge(df_alter, on='Match ', how = 'inner')
df_alt_merge['Alt_WIN'] = np.where(df_alt_merge['Score1 '] > df_alt_merge['Alt_Score'], 1, 0)
df_alt_merge['Alt_WIN'] = np.where(df_alt_merge['Score1 '] == df_alt_merge['Alt_Score'], 0.5, df_alt_merge['Alt_WIN'])

# remove rows where Team1 = Alt_Team
df_alt_merge = df_alt_merge[df_alt_merge['Team1 '] != df_alt_merge['Alt_Team']]

df_alt_wins = pd.pivot_table(df_alt_merge, values='Alt_WIN', index=['Team1 '], columns=['Alt_Team'], aggfunc='sum')

# divide by number of games played between each team
df_alt_wins = df_alt_wins.div(pd.pivot_table(df_alt_merge, values='Alt_WIN', index=['Team1 '], columns=['Alt_Team'], aggfunc='count'), axis=1)

# round up to 2 decimal places
df_alt_wins = df_alt_wins.round(2)

# Fill NaN values with -1 to indicate "not applicable"
df_alt_wins = df_alt_wins.fillna(-1)

# subtract df_alt_wins from df_wins
df_diff = df_wins - df_alt_wins

df_diff['Strength'] = df_diff.mean(axis=1)

# add rank column
df_diff['Rank'] = df_diff['Strength'].rank(ascending=False).astype(int)

# Fill NaN values with -1 to indicate "not applicable"
df_diff = df_diff.fillna(-1)

# Sort teams alphabetically
teams = sorted([col for col in df_diff.columns if col not in ['Strength', 'Rank']])

# Reorder columns to match sorted teams (plus Strength and Rank at the end)
df_diff = df_diff[teams + ['Strength', 'Rank']]

# Format the data for frontend consumption
matrix_data = df_diff.reset_index().to_dict('records')

matrix_data_wins = df_wins.reset_index().to_dict('records')

In [12]:
df_wins

Team2,Atlanta Hawks,Boston Celtics,Brooklyn Nets,Charlotte Hornets,Chicago Bulls,Cleveland Cavaliers,Dallas Mavericks,Denver Nuggets,Detroit Pistons,Golden State Warriors,Houston Rockets,Indiana Pacers,Los Angeles Clippers,Los Angeles Lakers,Memphis Grizzlies,Miami Heat,Milwaukee Bucks,Minnesota Timberwolves,New Orleans Pelicans,New York Knicks,Oklahoma City Thunder,Orlando Magic,Philadelphia 76ers,Phoenix Suns,Portland Trail Blazers,Sacramento Kings,San Antonio Spurs,Toronto Raptors,Utah Jazz,Washington Wizards
Team1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Atlanta Hawks,-1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,1.0,0.0,1.0,-1.0,-1.0,-1.0
Boston Celtics,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,-1.0,1.0,-1.0,0.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,0.0,0.0,0.0,-1.0,-1.0
Brooklyn Nets,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,0.0,-1.0,0.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,0.0,0.0,-1.0,-1.0,0.0,-1.0,-1.0,0.0,1.0,1.0
Charlotte Hornets,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,-1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0
Chicago Bulls,0.0,1.0,-1.0,1.0,-1.0,1.0,1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,0.0,0.0,-1.0,1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0
Cleveland Cavaliers,0.0,1.0,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,0.0,0.0,1.0,0.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0
Dallas Mavericks,-1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,-1.0,0.0,0.0,1.0,-1.0,1.0,-1.0,0.0,1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,0.0,-1.0,-1.0,-1.0
Denver Nuggets,0.0,-1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,0.0,0.0,0.0,1.0,-1.0,-1.0,-1.0,0.0,-1.0,1.0,-1.0,-1.0,-1.0
Detroit Pistons,0.0,1.0,-1.0,1.0,-1.0,1.0,1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,0.0,1.0,1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [24]:
import numpy as np

# Step 1: sampling grid
X_values = np.array([3, 4, 5, 6, 7, 8])
Y_values = np.linspace(-5, 5, 201)

# Prepare arrays to store (x_i, y_i) and the target f_i
x_data = []
y_data = []
f_data = []

# Step 2: compute F(X, Y) and collect data
for X in X_values:
    for Y in Y_values:
        F_val = ((X - 3) / 4.0) * np.tanh(X * Y)
        x_data.append(X)
        y_data.append(Y)
        f_data.append(F_val)

x_data = np.array(x_data)
y_data = np.array(y_data)
f_data = np.array(f_data)

# Step 3: Build design matrix
# Each row is [X, Y], corresponding to A*X + B*Y
M = np.column_stack((x_data, y_data))

# Step 4: Least-squares solution for A and B
# Solve M * [A, B]^T = f_data
params, residuals, rank, s = np.linalg.lstsq(M, f_data, rcond=None)
A_est, B_est = params

print("Estimated A =", A_est)
print("Estimated B =", B_est)


Estimated A = 7.88823465070619e-18
Estimated B = 0.1864140259717622


In [27]:
import numpy as np

def F(X, Y):
    return ((X - 3) / 4.0) * np.tanh(X * Y)

# 1. Create grid
X_values = np.array([3, 4, 5, 6, 7, 8])
Y_values = np.linspace(-1, 1, 201)  # 201 points in [-1,1]

# 2. Compute (x, y, f) data
x_data = []
y_data = []
f_data = []

for X in X_values:
    for Y in Y_values:
        x_data.append(X)
        y_data.append(Y)
        f_data.append(F(X, Y))

x_data = np.array(x_data)
y_data = np.array(y_data)
f_data = np.array(f_data)

# 3. Design matrix: each row = [X, Y]
M = np.column_stack((x_data, y_data))

# 4. Solve for [A, B] in A*X + B*Y ~ F(X,Y)
params, residuals, rank, s = np.linalg.lstsq(M, f_data, rcond=None)
A_est, B_est = params

print("A =", A_est)
print("B =", B_est)


A = 0.0
B = 0.9133381183606306


In [28]:
Y = 0.01

for X in X_values:
    print(X,  np.tanh(X * Y))

3 0.029991003238820143
4 0.03997868031116357
5 0.04995837495787998
6 0.059928103529143496
7 0.06988589031642899
8 0.07982976911113136


In [5]:
5*Y

3.5999999999999996

In [12]:
# Step 3 with intercept
M = np.column_stack((np.ones_like(x_data), x_data, y_data))

# Step 4
params, residuals, rank, s = np.linalg.lstsq(M, f_data, rcond=None)
C_est, A_est, B_est = params

print("C =", C_est, " A =", A_est, " B =", B_est)


C = -7.633203879574102e-17  A = 1.3548995448620395e-17  B = 0.9133381183606303


In [16]:
import numpy as np

def F(X, Y):
    return ((X - 3) / 4.0) * np.tanh(X * Y)

# Suppose you have a single Y of interest:
Y_interest = 0.5  # Example

# Discrete X values
X_values = [3, 4, 5, 6, 7, 8]

# Let's say the "true" X for this scenario is 5
X_true = 5

# Calculate F(X, Y_interest) for all possible X
F_at_Y = [F(X, Y_interest) for X in X_values]

# Look up the function value at the true X
F_true = F(X_true, Y_interest)

# Compare F_true against each X
for X in X_values:
    diff = F_true - F(X, Y_interest)
    print(f"X={X}: F({X_true},{Y_interest}) - F({X},{Y_interest}) = {diff:.4f}")


X=3: F(5,0.5) - F(3,0.5) = 0.4933
X=4: F(5,0.5) - F(4,0.5) = 0.2523
X=5: F(5,0.5) - F(5,0.5) = 0.0000
X=6: F(5,0.5) - F(6,0.5) = -0.2530
X=7: F(5,0.5) - F(7,0.5) = -0.5049
X=8: F(5,0.5) - F(8,0.5) = -0.7559


In [17]:
F(3, 0.5)

np.float64(0.0)

In [18]:
F(5, 0.5)

np.float64(0.49330714907571516)

In [21]:
import numpy as np

def F_func(X, Y):
    """
    Computes F(X,Y) = ((X - 3)/4)*tanh(X*Y).
    """
    return ((X - 3)/4.0) * np.tanh(X * Y)

# Example data (replace these with your actual arrays).
# Must have the same length for X_values and Y_values.
X_values = np.array([3, 4, 5, 6, 5, 7, 8, 3, 6, 7])
Y_values = np.array([0.2, 0.3, -0.7, 0.5, 0.5, -0.9, 1.0, -0.4, 0.3, 0.99])

# 1. Compute F_data for each (X, Y)
F_data = np.array([F_func(x, y) for x, y in zip(X_values, Y_values)])

# 2. Build design matrix for linear model without intercept: A*X + B*Y
M = np.column_stack((X_values, Y_values))

# 3. Solve M * [A, B]^T ≈ F_data
params, residuals, rank, s = np.linalg.lstsq(M, F_data, rcond=None)
A_est, B_est = params

# 4. Print results
print("Estimated coefficient A =", A_est)
print("Estimated coefficient B =", B_est)

# (Optional) Compute and show the root mean squared error
preds = M.dot(params)
rms_error = np.sqrt(np.mean((F_data - preds)**2))
print("Root Mean Squared Error =", rms_error)


Estimated coefficient A = 0.020959206088787777
Estimated coefficient B = 0.9883440589478584
Root Mean Squared Error = 0.20404635061408238


In [2]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# Coaching career data with example logos
data = [
    {"Club": "Ilisiakos", "Country": "Greece", "Start": "2002-07-01", "End": "2004-06-30", "Logo": "https://upload.wikimedia.org/wikipedia/en/3/38/Ilisiakos_logo.png"},
    {"Club": "AEK Athens", "Country": "Greece", "Start": "2004-07-01", "End": "2005-06-30", "Logo": "https://upload.wikimedia.org/wikipedia/en/thumb/f/f4/AEK_Athens_FC.svg/1200px-AEK_Athens_FC.svg.png"},
    {"Club": "Larissa", "Country": "Greece", "Start": "2007-01-01", "End": "2008-12-31", "Logo": "https://upload.wikimedia.org/wikipedia/en/9/92/AE_Larissa_logo.svg"},
    {"Club": "Atromitos", "Country": "Greece", "Start": "2009-01-01", "End": "2012-06-30", "Logo": "https://upload.wikimedia.org/wikipedia/en/8/8c/Atromitos_FC_logo.svg"},
    {"Club": "PAOK", "Country": "Greece", "Start": "2012-07-01", "End": "2013-04-30", "Logo": "https://upload.wikimedia.org/wikipedia/en/3/3e/PAOK_FC_logo.svg"},
    {"Club": "APOEL", "Country": "Cyprus", "Start": "2013-05-01", "End": "2015-01-06", "Logo": "https://upload.wikimedia.org/wikipedia/en/3/3c/APOEL_FC.svg"},
    {"Club": "Al-Hilal", "Country": "Saudi Arabia", "Start": "2015-05-25", "End": "2016-02-21", "Logo": "https://upload.wikimedia.org/wikipedia/en/4/44/Al-Hilal_FC_logo.svg"},
    {"Club": "Sharjah", "Country": "UAE", "Start": "2017-01-01", "End": "2017-06-01", "Logo": "https://upload.wikimedia.org/wikipedia/en/b/b1/Sharjah_FC_Logo.png"},
    {"Club": "Panathinaikos", "Country": "Greece", "Start": "2018-05-19", "End": "2020-06-30", "Logo": "https://upload.wikimedia.org/wikipedia/en/e/eb/Panathinaikos_FC_logo.svg"},
    {"Club": "Maccabi Tel Aviv", "Country": "Israel", "Start": "2020-10-28", "End": "2021-06-30", "Logo": "https://upload.wikimedia.org/wikipedia/en/f/fb/Maccabi_tel_aviv_fc.svg"},
    {"Club": "Al-Wehda", "Country": "Saudi Arabia", "Start": "2021-07-01", "End": "2022-06-30", "Logo": "https://upload.wikimedia.org/wikipedia/en/3/33/Al-Wehda_Club_logo.svg"},
]

df = df.sort_values("Start")
df["Midpoint"] = df["Start"] + (df["End"] - df["Start"]) / 2

fig = px.timeline(
    df,
    x_start="Start",
    x_end="End",
    y="Club",
    color="Country",
    hover_data=["Country", "Start", "End"],
    category_orders={"Club": df["Club"].tolist()}  # sort y-axis by start date
)

fig.update_yaxes(autorange="reversed")

# Add logos as images
for _, row in df.iterrows():
    fig.add_layout_image(
        dict(
            source=row["Logo"],
            x=row["Midpoint"],
            y=row["Club"],
            xref="x",
            yref="y",
            sizex=(row["End"] - row["Start"]).days / 5,
            sizey=0.4,
            xanchor="center",
            yanchor="middle",
            sizing="contain",
            layer="above"
        )
    )

fig.update_layout(
    template="plotly_dark",
    title="👔 Georgios Donis – Coaching Career Timeline",
    title_font_size=26,
    font=dict(family="Helvetica Neue", size=14),
    height=700,
    margin=dict(l=70, r=70, t=100, b=50),
    plot_bgcolor="#111",
    paper_bgcolor="#111",
    legend_title="",
    hoverlabel=dict(bgcolor="#222", font_size=13, font_family="Arial")
)

fig.show()
